In [37]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta,date
from numba import jit

# 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
sns.set_palette('icefire_r', 2)
import plotly.express as px
import plotly.io as pio
from plotly.io import show
import plotly.graph_objs as go

import warnings

%matplotlib inline
#Przetwarzanie

# Baza danych gównych do nauki modelu
train = pd.read_csv("train.csv") 
# Baza danych Titanica do sprawdzenia modelu
test = pd.read_csv("test.csv")

In [38]:
# print(pd.isnull(train).sum())

In [39]:
# print(pd.isnull(test).sum())

In [40]:
# cols= ['country', 'store', 'product' ]
# for col in cols:
#   df_grouped = train.groupby(col)[col].count()
#   df_grouped = df_grouped / train[col].count() * 100
#   print(f"Unique value counts and percentages for column {col}:")
#   print(df_grouped)

In [41]:
# cols= ['country', 'store', 'product' ]
# for col in cols:
#   df_grouped = test.groupby(col)[col].count()
#   df_grouped = df_grouped / test[col].count() * 100
#   print(f"Unique value counts and percentages for column {col}:")
#   print(df_grouped)

In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        136950 non-null  int64 
 1   date      136950 non-null  object
 2   country   136950 non-null  object
 3   store     136950 non-null  object
 4   product   136950 non-null  object
 5   num_sold  136950 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 6.3+ MB


In [43]:
train.index

RangeIndex(start=0, stop=136950, step=1)

In [44]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.month
train['day'] = pd.to_datetime(train['date']).dt.day
test['year'] = pd.to_datetime(test['date']).dt.year
test['month'] = pd.to_datetime(test['date']).dt.month
test['day'] = pd.to_datetime(test['date']).dt.day

In [45]:
train = train.set_index('date') 
test = test.set_index('date')

train.index = train.index.to_period(freq='D')

In [46]:
train.index

PeriodIndex(['2017-01-01', '2017-01-01', '2017-01-01', '2017-01-01',
             '2017-01-01', '2017-01-01', '2017-01-01', '2017-01-01',
             '2017-01-01', '2017-01-01',
             ...
             '2021-12-31', '2021-12-31', '2021-12-31', '2021-12-31',
             '2021-12-31', '2021-12-31', '2021-12-31', '2021-12-31',
             '2021-12-31', '2021-12-31'],
            dtype='period[D]', name='date', length=136950)

In [47]:
# fig = sns.lineplot(train, x=train.index.to_timestamp() , y='num_sold', hue=train['country']=='Spain')
# fig.show()

In [48]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
Enc_col = ['country', 'store', 'product' ]

# Fit the encoder and create the mapping for each column separately
categories = {}
for col in Enc_col:
    for df in [train, test]:
        df[col + '_Enc'] = enc.fit_transform(df[col].values.reshape(-1, 1))
    categories[col] = enc.categories_[0]  # Store categories for the current column

# Print the mapping
for col in Enc_col:
    print(f"Kolumna: {col}")
    for i, category in enumerate(categories[col]):
        print(f"  Wartość zakodowana: {i} -> Wartość początkowa: {category}")

Kolumna: country
  Wartość zakodowana: 0 -> Wartość początkowa: Argentina
  Wartość zakodowana: 1 -> Wartość początkowa: Canada
  Wartość zakodowana: 2 -> Wartość początkowa: Estonia
  Wartość zakodowana: 3 -> Wartość początkowa: Japan
  Wartość zakodowana: 4 -> Wartość początkowa: Spain
Kolumna: store
  Wartość zakodowana: 0 -> Wartość początkowa: Kagglazon
  Wartość zakodowana: 1 -> Wartość początkowa: Kaggle Learn
  Wartość zakodowana: 2 -> Wartość początkowa: Kaggle Store
Kolumna: product
  Wartość zakodowana: 0 -> Wartość początkowa: Using LLMs to Improve Your Coding
  Wartość zakodowana: 1 -> Wartość początkowa: Using LLMs to Train More LLMs
  Wartość zakodowana: 2 -> Wartość początkowa: Using LLMs to Win Friends and Influence People
  Wartość zakodowana: 3 -> Wartość początkowa: Using LLMs to Win More Kaggle Competitions
  Wartość zakodowana: 4 -> Wartość początkowa: Using LLMs to Write Better


In [49]:
train[['country_Enc', 'store_Enc', 'product_Enc']] = train[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
test[['country_Enc', 'store_Enc', 'product_Enc']] = test[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
train['num_sold'] = train['num_sold'].astype(np.int16)

In [50]:
categorical_features = ['id','country', 'store', 'product' ]
for df in [train, test]:
     df.drop(columns=categorical_features, inplace=True)

In [51]:
train.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 136950 entries, 2017-01-01 to 2021-12-31
Freq: D
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   num_sold     136950 non-null  int16
 1   year         136950 non-null  int32
 2   month        136950 non-null  int32
 3   day          136950 non-null  int32
 4   country_Enc  136950 non-null  int8 
 5   store_Enc    136950 non-null  int8 
 6   product_Enc  136950 non-null  int8 
dtypes: int16(1), int32(3), int8(3)
memory usage: 3.3 MB


In [52]:
train.head()

,num_sold,year,month,day,country_Enc,store_Enc,product_Enc
date,,,,,,,
2017-01-01,63,2017,1,1,0,1,0
2017-01-01,66,2017,1,1,0,1,1
2017-01-01,9,2017,1,1,0,1,2
2017-01-01,59,2017,1,1,0,1,3
2017-01-01,49,2017,1,1,0,1,4


In [53]:
from sklearn.preprocessing import MinMaxScaler

train.head()
scaler= MinMaxScaler()
# -------------------------------------------------------------
train_num_sol_reshaped = train['num_sold'].values.reshape(-1, 1)  # Reshape to 2D
scaler.fit(train_num_sol_reshaped)
train['num_sold_scaled'] = scaler.transform(train_num_sol_reshaped)
#----------------------------------------------------------------
train.drop('num_sold', axis=1, inplace=True)

In [60]:
from sklearn.model_selection import train_test_split

Spain = train[train['country_Enc'] == 4]
train_Spain, val_Spain = train_test_split(Spain, test_size=0.25, shuffle=False)

# Create a figure with subplots
fig = go.Figure(layout=dict(xaxis_title="Date", yaxis_title="Sales"))

# Plot train_Spain data in first subplot
trace1 = go.Scatter(
    x=train_Spain.index.to_timestamp(),
    y=train_Spain['num_sold_scaled'],
    name="Train",
  #  mode="markers",
    marker=dict(color="blue", size=8),
    mode="spline",
    line=dict(color="blue", width=2)

)

# Plot val_Spain data in second subplot
trace2 = go.Scatter(
    x=val_Spain.index.to_timestamp(),
    y=val_Spain['num_sold_scaled'],
    name="Validation",
    mode="markers",
    marker=dict(color="red", size=8)
)

# Add traces to the figure
fig.add_trace(trace1)
fig.add_trace(trace2)

# Display the chart
fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'mode' property of scatter
        Received value: 'spline'

    The 'mode' property is a flaglist and may be specified
    as a string containing:
      - Any combination of ['lines', 'markers', 'text'] joined with '+' characters
        (e.g. 'lines+markers')
        OR exactly one of ['none'] (e.g. 'none')

In [55]:
train_Spain.tail()

,year,month,day,country_Enc,store_Enc,product_Enc,num_sold_scaled
date,,,,,,,
2020-10-01,2020,10,1,4,1,2,0.006531
2020-10-01,2020,10,1,4,1,3,0.039187
2020-10-01,2020,10,1,4,1,4,0.025399
2020-10-01,2020,10,1,4,2,0,0.080552
2020-10-01,2020,10,1,4,2,1,0.074020


In [56]:
val_Spain.head()

,year,month,day,country_Enc,store_Enc,product_Enc,num_sold_scaled
date,,,,,,,
2020-10-01,2020,10,1,4,2,2,0.009434
2020-10-01,2020,10,1,4,2,3,0.058781
2020-10-01,2020,10,1,4,2,4,0.047170
2020-10-01,2020,10,1,4,0,0,0.302612
2020-10-01,2020,10,1,4,0,1,0.277213


In [57]:
----

SyntaxError: invalid syntax (2133496677.py, line 1)

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from statsmodels.tsa.api import SARIMAX


# Assuming 'train' is your DataFrame with 'num_sold_scaled' as the target variable
det_proc = DeterministicProcess(train.index, constant=False, fourier=3 )
det_proc.in_sample()

train_with_exo = pd.concat([train['num_sold_scaled'], det_proc.in_sample()], axis=1)
model = SARIMAX(train_with_exo['num_sold_scaled'],
                     order=(1, 1, 1),  # Adjust order parameters as needed
                     seasonal_order=(1, 1, 1, 12),  # Adjust seasonal order as needed
                     exog=train_with_exo[['sin(1,7)', 'cos(1,7)',	'sin(2,7)',	'cos(2,7)',	'sin(3,7)',	'cos(3,7)']])

SARIMAX_fit = model.fit()

In [ ]:
fig = px.area(train, x=train.index.to_timestamp() , y='num_sold_scaled', template="simple_white")
fig.show()

In [ ]:
----

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam  # Import Adam optimizer

from tqdm import tqdm
torch.set_num_threads(12)


In [ ]:
features = ['year', 'month', 'day', "country_Enc", "store_Enc", "product_Enc"]
target   = 'num_sold_scaled'

# Przygotuj dane dla modelu
X_train = train[features]
y_train = train[target]

X_test = test[features]

# Konwertuj dane na tensory PyTorch
X_tensor_train = torch.from_numpy(X_train.to_numpy()).float()
y_tensor_train = torch.from_numpy(y_train.to_numpy()).float().reshape(-1, 1)

# Konwertuj dane na tensory PyTorch
X_tensor_test = torch.from_numpy(X_test.to_numpy()).float()

class MyDataset(torch.utils.data.Dataset):
  def __init__(self, X_tensor, y_tensor):
    if y_tensor is not None:
      # Handle missing labels (e.g., assign a specific value)
      self.y_tensor = [label if label is not None else -1 for label in y_tensor]
    self.X_tensor = X_tensor
    self.y_tensor = y_tensor

  def __getitem__(self, idx):
    X = self.X_tensor[idx]
    y = self.y_tensor[idx]
    return X, y

  def __len__(self):
    return len(self.X_tensor)

train_dataset = MyDataset(X_tensor_train, y_tensor_train)
test_dataset = MyDataset(X_tensor_test, None) 

train_size = int(0.20 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create a DataLoader for batch training
batch_size = 2048 # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(LSTMModel, self).__init__()
    self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=0.2)  # Add dropout for regularization
    self.fc = nn.Linear(hidden_dim, output_dim)
    self.silu = nn.SiLU()  # Add SiLU activation layer

  def forward(self, x):
    # Pass the input through LSTM layers
    x, (hidden, cell) = self.lstm(x)
    # Use the output from the last hidden layer
    x = hidden[-1]
    # Apply SiLU activation before linear layer
    x = self.silu(x)
    # Apply linear layer for prediction
    x = self.fc(x)
    return x

# Hyperparameters (adjust as needed)
input_dim = len(features)  # Number of features
hidden_dim =256
num_layers = 4
output_dim = 1  # Assuming predicting single value (num_sold)

model = LSTMModel(input_dim, hidden_dim, output_dim).float()
criterion = nn.HuberLoss()
optimizer = torch.optim.NAdam(model.parameters(), lr=0.005)  # Set learning rate

# Training loop
num_epochs = 36
best_loss = float('inf')
print(model)
loss_history = []  # Initialize a list to store loss values

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    pbar = tqdm(train_loader)  # Wrap train_loader with tqdm
    for batch_x, batch_y in pbar:
        optimizer.zero_grad()
        for x, y in zip(batch_x, batch_y):
            x = x.unsqueeze(0)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

        # Update progress bar
        pbar.set_description(f"Epoch {epoch+1}/{num_epochs} Loss: {loss.item():.2f}")

        # Append loss to history
        loss_history.append(loss.item())

    # Plot loss history after each epoch
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history, label='Training Loss')
    plt.title(f"Training Loss (Epochs: {num_epochs})")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend()
    plt.show()